<a href="https://colab.research.google.com/github/debi201326/AAI_Practical/blob/main/AAI_prac_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Practical 2:  Design a hierarchical planning problem and develop a hierarchical planner to solve it.

In [ ]:
class HierarchicalPlanner:
    def __init__(self, grid, start, warehouse, destination):
        self.grid = grid  # 2D grid environment
        self.position = start  # Robot's start position
        self.warehouse = warehouse  # Package pickup location
        self.destination = destination  # Delivery location
        self.package_picked = False  # Status of package

    def high_level_plan(self):
        """Defines the high-level plan."""
        return ["Navigate to Warehouse", "Pick Up Package", "Navigate to Destination", "Deliver Package"]

    def mid_level_plan(self, goal):
        """Breaks a high-level task into a sequence of movements."""
        path = self.a_star_search(self.position, goal)
        return ["Move to " + str(step) for step in path]

    def a_star_search(self, start, goal):
        """A* search for pathfinding."""
        from queue import PriorityQueue
        def heuristic(a, b):
            return abs(a[0] - b[0]) + abs(a[1] - b[1])

        open_set = PriorityQueue()
        open_set.put((0, start))
        came_from = {}
        g_score = {start: 0}
        f_score = {start: heuristic(start, goal)}

        while not open_set.empty():
            _, current = open_set.get()
            if current == goal:
                path = []
                while current in came_from:
                    path.append(current)
                    current = came_from[current]
                return path[::-1]

            for dx, dy in [(0,1), (1,0), (0,-1), (-1,0)]:
                neighbor = (current[0] + dx, current[1] + dy)
                if neighbor in self.grid and self.grid[neighbor] != 1:
                    temp_g_score = g_score[current] + 1
                    if neighbor not in g_score or temp_g_score < g_score[neighbor]:
                        came_from[neighbor] = current
                        g_score[neighbor] = temp_g_score
                        f_score[neighbor] = temp_g_score + heuristic(neighbor, goal)
                        open_set.put((f_score[neighbor], neighbor))
        return []

    def execute_plan(self):
        """Executes the hierarchical plan step by step."""
        for task in self.high_level_plan():
            print(f"Executing: {task}")

            if task == "Navigate to Warehouse":
                for step in self.mid_level_plan(self.warehouse):
                    print(f"  {step}")
                    self.position = eval(step.split("Move to ")[1])
            elif task == "Pick Up Package":
                self.package_picked = True
                print("  Package Picked Up")
            elif task == "Navigate to Destination":
                for step in self.mid_level_plan(self.destination):
                    print(f"  {step}")
                    self.position = eval(step.split("Move to ")[1])
            elif task == "Deliver Package":
                if self.position == self.destination and self.package_picked:
                    print("  Package Delivered Successfully!")
                else:
                    print("  Failed to Deliver Package")

# Define the grid environment (0 = free space, 1 = obstacle)
grid = {(x, y): 0 for x in range(5) for y in range(5)}  # 5x5 empty grid
start = (0, 0)
warehouse = (2, 2)
destination = (4, 4)

# Create and execute the hierarchical planner
planner = HierarchicalPlanner(grid, start, warehouse, destination)
planner.execute_plan()
